## Introduction à la quantization 

Laurent cetinsoy

Les réseaux de neurones prennent beaucoup de place et il peut être difficile de les faire rentrer sur certains dispositifs embarqués. 

Il existe plusieurs méthodes pour réduire la taille et augmenter la vitesse d'executer des réseaux de neurone. Par exemple il y a ce qu'on appelle la quantization et le pruning.

Dans ce notebook on va faire une introduction à la quantization avec la librairie tensorflow lite.


## Quantization post training

Dans un premier temps on va quantifier notre réseau après l'avoir entraîné normalement. 


Entraîner un réseau de neurone convolutionnel simple avec keras pour faire de la classification MNIST (ou un autre dataset simple de votre choix si (vous en avez marre de ce dataset - https://keras.io/api/datasets/)




In [ ]:
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = load_data()

x_train = x_train.reshape(60000, 28, 28, 1) / 255
x_train.shape
x_test = x_test.reshape(-1, 28, 28, 1) / 255
x_train.shape


model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(200, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_test, y_test))

1875/1875 [==============================] - 211s 112ms/step - loss: 0.1040 - accuracy: 0.9679 - val_loss: 0.0460 - val_accuracy: 0.9849


Afficher le nombre de paramètre du modèle

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 9216)              0         
                                                                 
 dense_2 (Dense)             (None, 200)               1843400   
                                                                 
 dense_3 (Dense)             (None, 10)                2010      
                                                      

Sauvegarder votre modèle et afficher la taille du fichier. Si on applique une bête règle de trois, quelle est la taille occupée par paramètre ? 

In [ ]:
model.save("model.h5")

import os

print("Taille du fichier: ",os.path.getsize("model.h5"), "octets" )

print("Taille occupée par parametre: ", os.path.getsize("model.h5")/model.count_params() , "octets" )

Taille du fichier:  22414000 octets
Taille occupée par parametre:  12.023220360621513 octets


On va maintenant convertir notre modèle keras en modèle tensorflow lite. 

Installer la librairie tensorflow lite créer une instance de la class TFLiteConverter à partir de votre modèle keras


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)

Convertir votre modèle et le sauvegarder dans un fichier nommé model.tflite. Sa taille est-elle plus petite ? 

In [ ]:
model_quantized = converter.convert()

with open('model_quantized.tflite', 'wb') as f:
  f.write(model_quantized)

import os

print("Taille du fichier Keras: ",os.path.getsize("model.h5"), "octets" )
print("Taille du fichier Tensorflow Lite: ",os.path.getsize("model_quantized.tflite"), "octets" )

Taille du fichier Keras:  22414000 octets
Taille du fichier Tensorflow Lite:  7460192 octets


On va maintenant spécifier des optimisations au converter. 

1. Recréer un converter

2. modifier son attribut optimizations pour ajouter une liste d'optimisation avec la valeur tf.lite.Optimize.DEFAULT

3. Relancer la conversion du modèle, sauvegarder le modèle et regarder la taille du fichier généré

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("model.h5")

converter_optimized = tf.lite.TFLiteConverter.from_keras_model(model)

converter_optimized.optimizations = [tf.lite.Optimize.DEFAULT]

model_quantized_optimized = converter_optimized.convert()

with open('model_quantized_optimized.tflite', 'wb') as f:
    f.write(model_quantized_optimized)
  
import os
print("Taille du modèle optimisé:", os.path.getsize('model_quantized_optimized.tflite'), "octets")

Taille du modèle optimisé: 1870216 octets


Quelle type  de quantization Optimize.Default, utilise-t-elle ?


L'optimisation tf.lite.Optimize.DEFAULT utilise une combinaison de quantification de poids et d'activation ainsi que d'autres optimisations telles que l'élagage et la fusion de couches. Cette combinaison d'optimisations peut réduire considérablement la taille du modèle et améliorer les performances d'inférence sur les appareils embarqués.

Plus précisément, lors de l'utilisation de tf.lite.Optimize.DEFAULT, le convertisseur TensorFlow Lite utilise par défaut la pondération 8 bits et active la quantification. Cela signifie que les pondérations et les activations du modèle sont converties en entiers 8 bits au lieu de flottants 32 bits, ce qui réduit considérablement la taille du modèle. Le convertisseur peut également nettoyer en supprimant les poids qui ont peu d'importance relative. Enfin, le convertisseur peut fusionner certaines couches du modèle pour réduire davantage la taille du modèle. 

## Quantization aware training 

Dans cette section on va s'intéresser à l'entraînement sensible à la quantification. L'idée est de simuler les effets de la quantification pendant l'entraînement pour que le modèle ajuste les poids afin de tenir ocmpte de la quantification. 

Reprendre le modèle entraîné sur MNIST


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("model.h5")

A l'aide de la fonction quantize de tensorflow_model_optimization, créer une seconde version de votre modèle entraîné nommé qat_model

In [ ]:
!pip install tensorflow-model-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow_model_optimization as tfmot

qat_model = tfmot.quantization.keras.quantize_model(model)

Compiler le modèle

In [ ]:
qat_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Afficher le summury du modèle. D'après vous ce modèle est-il quantifié ? 

In [ ]:
qat_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_1 (QuantizeL  (None, 28, 28, 1)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_2 (QuantizeWra  (None, 26, 26, 32)       387       
 pperV2)                                                         
                                                                 
 quant_conv2d_3 (QuantizeWra  (None, 24, 24, 64)       18627     
 pperV2)                                                         
                                                                 
 quant_max_pooling2d_1 (Quan  (None, 12, 12, 64)       1         
 tizeWrapperV2)                                                  
                                                                 
 quant_flatten_2 (QuantizeWr  (None, 9216)            

Réentraîner votre modèle sur un sous ensemble des modèles sur une ou deux epochs et afficher la performance sur le train et test set

In [ ]:
qat_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

loss, accuracy = qat_model.evaluate(x_test, y_test)

print("Test set accuracy: {:.2f}%".format(accuracy * 100))

loss, accuracy = qat_model.evaluate(x_train, y_train)

print("Training set accuracy: {:.2f}%".format(accuracy * 100))

Epoch 1/2
1875/1875 [==============================] - 266s 139ms/step - loss: 0.0385 - accuracy: 0.9879 - val_loss: 0.0338 - val_accuracy: 0.9891
Epoch 2/2
313/313 [==============================] - 10s 32ms/step - loss: 0.0302 - accuracy: 0.9911
Test set accuracy: 99.11%
1875/1875 [==============================] - 60s 32ms/step - loss: 0.0081 - accuracy: 0.9976
Training set accuracy: 99.76%


Convertir votre modèle avec TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

qat_model_quantized_optimized = converter.convert()

with open('qat_model_quantized.tflite', 'wb') as f:
  f.write(qat_model_quantized_optimized)

/usr/local/lib/python3.9/dist-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Convertir le model en .h pour le mettre sur l'Arduino

In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat qat_model_quantized.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h


Comparer la performance du modèle Quantified aware training, au modèle original et au modèle quantifié post training

In [ ]:
import numpy as np

In [ ]:
def evaluate_tflite(tflite_model):

  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details() 

  y_pred = []

  for i in range(len(x_test)):
      input_data = np.expand_dims(x_test[i], axis=0).astype(input_details[0]['dtype'])
      interpreter.set_tensor(input_details[0]['index'], input_data)
      interpreter.invoke()
      output = interpreter.tensor(output_details[0]['index'])
      y_pred.append(np.argmax(output()[0]))

  # Comparaison entre la prédiction et la vraie valeur de test
  y_pred = np.array(y_pred)
  accuracy = (y_pred == y_test).mean()

  return accuracy

In [ ]:
# Evaluation du modèle Quantified aware training
quantized_aware_train_accuracy = evaluate_tflite(qat_model_quantized_optimized)

# Evaluation du modèle original
original_loss, original_accuracy = model.evaluate(x_test, y_test)

# Evaluation du modèle Post training
post_training_accuracy = evaluate_tflite(model_quantized_optimized)

print("Performance du modèle Quantified aware training : accuracy = {:.4f}".format(quantized_aware_train_accuracy))
print("Performance du modèle original : loss = {:.4f}, accuracy = {:.4f}".format(original_loss, original_accuracy))
print("Performance du modèle Post training : accuracy = {:.4f}".format(post_training_accuracy))

313/313 [==============================] - 9s 28ms/step - loss: 0.0460 - accuracy: 0.9849
Performance du modèle Quantified aware training : accuracy = 0.9911
Performance du modèle original : loss = 0.0460, accuracy = 0.9849
Performance du modèle Post training : accuracy = 0.9849


Sauvegarder le modèle QAT et comparer les tailles des modèles

In [ ]:
qat_model.save("qat_model.h5")

import os

print("Taille du modèle Quantified aware training : ", os.path.getsize('qat_model_quantized.tflite'), "octets")
print("Taille du modèle original : ", os.path.getsize('model.h5'), "octets")
print("Taille du modèle Post training:", os.path.getsize('model_quantized_optimized.tflite'), "octets")

Taille du modèle Quantified aware training :  1872256 octets
Taille du modèle original :  22414000 octets
Taille du modèle Post training: 1870216 octets


Bonus : déployer votre modèle sur votre téléphone ou un dispositif embarqué si vous en disposez d'un. 

Bonus : Obtenir un modèle qui sera à la fois quantifié et élagué (prunned) en s'aidant de la documentation (https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras)

A l'aide de tensorflow lite / tensorflow lite micro 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0d51e245-899d-41d6-b23b-cf3e4bbbc6ea' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>